In [1]:
from gigachat_api import GigaChat

gigachat = GigaChat()

In [2]:
import pandas as pd

filename = 'data1.xlsx'
df = gigachat.process(filename)
df

2\11

/home/prof_evaluation/miniconda3/envs/mass/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'ngw.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(
/home/prof_evaluation/miniconda3/envs/mass/lib/python3.9/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'gigachat.devices.sberbank.ru'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


,chunk,keywords
0,"Привет ! Меня зовут Владислав Ефимов , я рабо...",1. Python\n2. IT-профессии\n3. Интерпретируемы...
1,А скомбинировав это с разработкой веб-приложе...,NaN
2,Давайте выведем стандартный для начала курсов...,NaN
3,Мы сейчас рассмотрели варианты разработки в и...,NaN
4,"Если мы хотим его сохранить , то можно это вы...",NaN
5,"Первая строка , она на самом деле аналогична ...",NaN
6,"Что значит этот код ? Значит , что в переменн...",NaN
7,"Для чего это нужно ? Это , во-первых , называ...",NaN
8,Потому что мы с этими данными ничего не делал...,NaN
9,И на самом деле запускается Python интерпрета...,NaN


In [3]:
import re
import itertools
data = []

for index, row in df.iterrows():
    keywords = re.split(r'\n\d+\.', row.keywords[3:].lower())
    new_keywords = []
    for key in keywords:
        tmp = re.split(r'\s+', re.sub(r'^\s+|\s+$', '', key))
        if len(tmp) > 1:
            perms = list(itertools.permutations(tmp))
            perms = [' '.join(items) for items in perms]
            new_keywords.extend(perms)
        else:
            new_keywords.append(tmp[0])

    data.append([re.split(r'\s+', row.chunk.lower()), new_keywords])
    break

data[0]

[['',
  'привет',
  '!',
  'меня',
  'зовут',
  'владислав',
  'ефимов',
  ',',
  'я',
  'работаю',
  'в',
  'команде',
  'вк',
  'рекламы',
  '.',
  'прежде',
  'чем',
  'приступить',
  'к',
  'изучению',
  'языка',
  ',',
  'я',
  'кратко',
  'расскажу',
  'о',
  'преимуществах',
  'этого',
  'языка',
  'программирования',
  ',',
  'какие',
  'задачи',
  'можно',
  'с',
  'помощью',
  'него',
  'решать',
  'и',
  'в',
  'каких',
  'it',
  '-',
  'профессиях',
  'он',
  'используется',
  '.',
  'начнем',
  'с',
  'отличительных',
  'преимуществ',
  'языка',
  '.',
  'во-первых',
  ',',
  'это',
  'простой',
  'для',
  'понимания',
  'использование',
  'язык',
  '.',
  'у',
  'языка',
  'python',
  ',',
  'пожалуй',
  ',',
  'один',
  'из',
  'самых',
  'простых',
  'и',
  'понятных',
  'синтаксисов',
  'относительно',
  'других',
  'языков',
  '.',
  'во-вторых',
  ',',
  'язык',
  'интерпретируемый',
  '.',
  'это',
  'значит',
  ',',
  'что',
  'вы',
  'можете',
  'написать',
  'код

In [4]:
pd.DataFrame(data=data, columns=['chunk', 'keywords']).to_excel('data_with_keywords.xlsx', index=False)

In [5]:
from sentence_splitter import SyntaxAnalyzer

sa = SyntaxAnalyzer()

/home/prof_evaluation/miniconda3/envs/mass/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-24 05:25:21 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


True


2023-11-24 05:25:22 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |

2023-11-24 05:25:22 INFO: Using device: cuda
2023-11-24 05:25:22 INFO: Loading: tokenize
2023-11-24 05:25:27 INFO: Loading: pos
2023-11-24 05:25:27 INFO: Loading: lemma
2023-11-24 05:25:28 INFO: Loading: depparse
2023-11-24 05:25:28 INFO: Done loading processors!
2023-11-24 05:25:29 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


['привет', 'дело', 'съесть дед']


2023-11-24 05:25:30 INFO: Loading these models for language: ru (Russian):
| Processor | Package            |
----------------------------------
| tokenize  | syntagrus          |
| pos       | syntagrus_charlm   |
| lemma     | syntagrus_nocharlm |
| depparse  | syntagrus_charlm   |

2023-11-24 05:25:30 INFO: Using device: cuda
2023-11-24 05:25:30 INFO: Loading: tokenize
2023-11-24 05:25:30 INFO: Loading: pos
2023-11-24 05:25:31 INFO: Loading: lemma
2023-11-24 05:25:31 INFO: Loading: depparse
2023-11-24 05:25:32 INFO: Done loading processors!


In [45]:
import ast
df = pd.read_excel('data_with_keywords.xlsx')

data = []
for index, row in df.iterrows():
    chunk = ast.literal_eval(row.chunk)

    bio_markup = []
    i = -1
    while i < len(chunk) - 1:
        i += 1
        if re.search(r'[A-Za-zА-ЯЁа-яё]', chunk[i]) is None:
            bio_markup.append('O')
            continue
        was_added = False
        for key in ast.literal_eval(row.keywords):

            # print(key)
            print(key, '|', chunk[i])
            parts = re.split(r'\s+', key)

            if len(parts) == 1:
                if chunk[i] == key:
                    bio_markup.append('B')
                    was_added=True
                    break
                else:
                    continue
                    # bio_markup.append('O')
            else:
                # print(len(chunk), i, len(parts))
                if i + len(parts) < len(chunk):
                    cur = []
                    if chunk[i] == parts[0]:
                        cur.append('B')
                    else:
                        continue
                    for j in range(1, len(parts)):
                        # print(parts)
                        # print(j)
                        # print(parts[j])
                        if chunk[i+j] == parts[j]:
                            cur.append('I')
                        else:
                            break
                    if len(cur) == len(parts):
                        i = i + len(parts) - 1
                        was_added=True
                        bio_markup.extend(cur)
                        break
                else:
                    continue
                    # bio_markup.append('O')
        if not was_added:
            bio_markup.append('O')

bio_markup


python | привет
it-профессии | привет
интерпретируемый язык | привет
язык интерпретируемый | привет
библиотеки | привет
портативность | привет
веб-разработка | привет
бэкэнд-разработка | привет
машинное обучение | привет
обучение машинное | привет
искусственный интеллект | привет
интеллект искусственный | привет
ml-специалист | привет
python | меня
it-профессии | меня
интерпретируемый язык | меня
язык интерпретируемый | меня
библиотеки | меня
портативность | меня
веб-разработка | меня
бэкэнд-разработка | меня
машинное обучение | меня
обучение машинное | меня
искусственный интеллект | меня
интеллект искусственный | меня
ml-специалист | меня
python | зовут
it-профессии | зовут
интерпретируемый язык | зовут
язык интерпретируемый | зовут
библиотеки | зовут
портативность | зовут
веб-разработка | зовут
бэкэнд-разработка | зовут
машинное обучение | зовут
обучение машинное | зовут
искусственный интеллект | зовут
интеллект искусственный | зовут
ml-специалист | зовут
python | владислав
it-профес

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B',
 'I',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O'

In [46]:
len(bio_markup), len(chunk)

(296, 296)

In [47]:
dict(zip(chunk, bio_markup))

{'': 'O',
 'привет': 'O',
 '!': 'O',
 'меня': 'O',
 'зовут': 'O',
 'владислав': 'O',
 'ефимов': 'O',
 ',': 'O',
 'я': 'O',
 'работаю': 'O',
 'в': 'O',
 'команде': 'O',
 'вк': 'O',
 'рекламы': 'O',
 '.': 'O',
 'прежде': 'O',
 'чем': 'O',
 'приступить': 'O',
 'к': 'O',
 'изучению': 'O',
 'языка': 'O',
 'кратко': 'O',
 'расскажу': 'O',
 'о': 'O',
 'преимуществах': 'O',
 'этого': 'O',
 'программирования': 'O',
 'какие': 'O',
 'задачи': 'O',
 'можно': 'O',
 'с': 'O',
 'помощью': 'O',
 'него': 'O',
 'решать': 'O',
 'и': 'O',
 'каких': 'O',
 'it': 'O',
 '-': 'O',
 'профессиях': 'O',
 'он': 'O',
 'используется': 'O',
 'начнем': 'O',
 'отличительных': 'O',
 'преимуществ': 'O',
 'во-первых': 'O',
 'это': 'O',
 'простой': 'O',
 'для': 'O',
 'понимания': 'O',
 'использование': 'O',
 'язык': 'O',
 'у': 'O',
 'python': 'B',
 'пожалуй': 'O',
 'один': 'O',
 'из': 'O',
 'самых': 'O',
 'простых': 'O',
 'понятных': 'O',
 'синтаксисов': 'O',
 'относительно': 'O',
 'других': 'O',
 'языков': 'O',
 'во-вторы

In [8]:
re.split(' ', 'privet')

['privet']